# Tâche #2 : Classification d'incidents avec un réseau  récurrent et des *embeddings* Spacy

Cette tâche est similaire à la précédente et vous réutilisez les mêmes fichiers d’entraînement, de validation et de test. Cependant, vous devez utiliser des réseaux récurrents pour classifier les textes. Plus particulièrement, vous devez entraîner un réseau de neurones LSTM pour encoder les textes et une couche linéaire pour faire la classification des textes. 

Les consignes pour cette tâche sont: 
- 	Nom du notebook : rnn.ipynb
- 	Tokenisation : Utilisation de Spacy. 
- 	Plongements de mots : Ceux de Spacy. 
- 	Normalisation : Aucune normalisation. 
- 	Structure du réseau : Un réseau LSTM avec 1 seule couche pour l’encodage de textes. Je vous laisse déterminer la taille de cette couche (à expliquer). 
- 	Analyse : Comparer les résultats obtenus avec un réseau unidirectionnel et un réseau bidirectionnel. Si vous éprouvez des difficultés à entraîner les 2 réseaux dans un même notebook, faites une copie et nommez le 2e fichier rnn-bidirectionnel.ipynb.
- 	Expliquez comment les modèles sont utilisés pour faire la classification d’un texte. 
- 	Présentez clairement vos résultats et faites-en l’analyse. 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [1]:
import json

def load(filename: str):
    dataset = {}
    with open(filename, "r") as fp:
        incident_list = json.load(fp)
    texts, labels = zip(*[(incident.get("text"), int(incident.get("label"))) for incident in incident_list])
    dataset["text"] = list(texts)
    dataset["label"] = list(labels)

    return dataset

In [2]:
dataset = load('./data/incidents_train.json')

#### 1.1 Conversion en Identifiants

In [3]:

labels = list(set(dataset["label"]))
id2label = {int(label): label for label in labels}
label2id = {label: int(label) for label in labels}
nb_labels = len(labels)

## 2. Gestion de plongements de mots (*embeddings*)

In [4]:
import spacy

model = spacy.load('en_core_web_md')

embedding_size = model.meta['vectors']['width']

In [5]:
from enum import Enum
import numpy as np

class Tokens(Enum):
    PADDING = "<PAD>"
    UNKNOWN = "<UNK>"

zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

id2word = {}
id2word[0] = Tokens.PADDING
id2word[1] = Tokens.UNKNOWN

word2id = {}
word2id[Tokens.PADDING] = 0
word2id[Tokens.UNKNOWN] = 1

id2embedding = {}
id2embedding[0] = zero_vec_embedding
id2embedding[1] = zero_vec_embedding


In [6]:
def map_ids():
    word_index = 2
    vocabulary = word2id.keys()

    for sentence in dataset["text"]:
        for word in model(sentence):
            if word.text not in vocabulary:
                word2id[word.text] = word_index
                id2word[word_index] = word.text
                id2embedding[word_index] = word.vector
                word_index += 1

map_ids()

In [7]:
import torch

from torch import LongTensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

class QuestionDataset(Dataset):
    def __init__(self, data , targets, word_to_id, spacy_model):
        self.data = data
        self.sequences = [None for _ in range(len(data))]
        self.targets = targets
        self.word2id = word_to_id
        self.tokenizer = spacy_model

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.sequences[index] is None:
            self.sequences[index] = self.tokenize(self.data[index])
        
        return LongTensor(self.sequences[index]), LongTensor([self.targets[index]]).squeeze(0)

    def tokenize(self, sentence):
        tokens = [word.text for word in self.tokenizer(sentence)]
        
        return [self.word2id.get(token, 1) for token in tokens] 


train_dataset = QuestionDataset(dataset['text'], dataset['label'], word2id, model)

dataset = load('./data/incidents_dev.json')

validation_dataset = QuestionDataset(dataset['text'], dataset['label'], word2id, model)

In [8]:
train_dataset[2]

(tensor([  2, 107,   8, 108,   2, 109,   2,  11,  12,  13,   2,  16, 110, 111,
         112, 113, 114, 115, 116, 117, 118,   2, 119, 120,  14, 121, 122,  32,
         123, 124,  16, 125, 126,  26,  66, 127,  32,  30, 128,   2, 129,  87,
         130, 131, 132,  53, 133, 134,  26,  66, 128, 129, 127,  14,   2, 135,
          30, 136,  32,  30, 129, 137,  56, 138,  30, 133,  34, 115,   2,  30,
         136,  26,  11,  12,  13, 139,  14, 140,  30, 141,  32,  30, 142,  87,
          16, 143,  32,  30,   2, 144, 145, 146,  30, 147, 148, 149, 150,  53,
          30, 136, 151,  14,   2, 152, 115, 153, 154,  26,  11,  12,  13,  14,
         155,  30,  83,   2, 151,  14,   2, 121, 156,  32,  30, 147, 148, 149,
         157,  30, 136, 138,  30, 158,   2, 134,  26,  11,  12,  13,  14, 159,
          62,  30, 160,  26,  66, 161, 140,   2, 162,  48, 163,  51, 164, 165,
           7,  59,  32,  30, 166, 167, 168, 151,  51,   2,   4, 169, 112, 170,
         171,  62, 172,  26, 173]),
 tensor(2))

## 3. Création de modèle(s)

In [69]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Rnn(nn.Module):
    def __init__(self, embeddings, hidden_state_size, nb_classes, bidirectional=False):
        super(Rnn, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        self.embedding_size = embeddings.size()[1]
        self.lstm = nn.LSTM(self.embedding_size, hidden_state_size, 1, batch_first=True, bidirectional=bidirectional)
        self.classification_layer = nn.Linear(hidden_state_size * 2 if bidirectional else hidden_state_size, nb_classes)
        self.hidden_state = None
        self.context = None

    def forward(self, x, x_lengths):
        x = self.embedding_layer(x)
        packed_batch = pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
        output, (self.hidden_state, self.context) = self.lstm(packed_batch)

        if self.lstm.bidirectional:
            output, _ = pad_packed_sequence(output, batch_first=True) 
            x = output.squeeze()
            x = self.classification_layer(output)
        else:
            x = self.hidden_state.squeeze()
            x = self.classification_layer(x)

        return x

In [70]:
def pad_batch(batch):
    x = [x for x,y in batch]
    x_true_length = [len(x) for x,y in batch]
    y = torch.stack([y for x,y in batch], dim=0)
    return ((pad_sequence(x, batch_first=True), x_true_length), y)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
validation_dataloader = DataLoader(validation_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)

In [71]:
vocab_size = len(id2embedding)

embedding_layer = np.zeros((vocab_size, embedding_size), dtype=np.float32)

for token_id, embedding in id2embedding.items():
    embedding_layer[token_id,:] = embedding
    
embedding_layer = torch.from_numpy(embedding_layer)

In [72]:
from poutyne import set_seeds

set_seeds(42)

hidden_size = 100 

model = Rnn(embedding_layer, hidden_size, nb_labels, True)

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

In [73]:
from poutyne.framework import Experiment
import torch.optim as optim
from torch.nn.functional import cross_entropy

learning_rate = 0.1

optimizer = optim.SGD(model.parameters(), learning_rate)

experiment = Experiment('model/lstm_classification', model, optimizer=optimizer, task="classification", loss_function=cross_entropy)

logging = experiment.train(train_dataloader, validation_dataloader, epochs=25, disable_tensorboard=True)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x100 and 200x9)

## 6. Évaluation et analyse de résultats